In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train','test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True)

def normalize_img(image, label):
    return tf.cast(image, tf.float32)/255.0, label

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

def augment(image, label):
    new_height = new_width = 32
    image = tf.image.resize(image, (new_height,new_width))
    
    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image),[1,1,3])
    
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
    
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_flip_up_down(image)
    
    return image, label

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
# ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

In [4]:
data_augmenation = keras.Sequential([
    layers.experimental.preprocessing.Resizing(height=32,width=32),
    layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    layers.experimental.preprocessing.RandomContrast(factor=0.1)
])

In [5]:
model = keras.Sequential([
    keras.Input((32,32,3)),
    data_augmenation,
    layers.Conv2D(4,3,padding='same',activation='relu'),
    layers.Conv2D(8,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='softmax')
])

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(3e-4),
    metrics=['accuracy'])

model.fit(ds_train,epochs=5)
model.evaluate(ds_test)

Epoch 1/5
1563/1563 [==============================] - 11s 5ms/step - loss: 1.8576 - accuracy: 0.3242
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.4678 - accuracy: 0.4763
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.3732 - accuracy: 0.5129
Epoch 4/5
1563/1563 [==============================] - 9s 5ms/step - loss: 1.3000 - accuracy: 0.5380
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 1.2247 - accuracy: 0.5618


[1.224692463874817, 0.5618000030517578]